# What is Corona Virus(COVID-19)?

**Coronavirus is a family of viruses that can cause illness, which can vary from common cold and cough to sometimes more severe disease. SARS-CoV-2 (n-coronavirus) is the new virus of the coronavirus family, which first discovered in 2019, which has not been identified in humans before. It is a contiguous virus which started from Wuhan in December 2019. Which later declared as Pandemic by WHO due to high rate spreads throughout the world. Currently (on date 27 March 2020), this leads to a total of 24K+ Deaths across the globe, including 16K+ deaths alone in Europe.Pandemic is spreading all over the world; it becomes more important to understand about this spread. This NoteBook is an effort to analyze the cumulative data of confirmed, deaths, and recovered cases over time. In this notebook, the main focus is to analyze the spread trend of this virus all over the india.**

# History of COVID-19 in India

**On January 30, India reported its first case of COVID-19 in Kerala, which rose to three cases by February 3; all were students who had returned from Wuhan, China.No significant rise in cases was seen in the rest of February.**

**On 22 March 2020, India observed a 14-hour voluntary public curfew at the instance of the prime minister Narendra Modi.The government followed it up with lockdowns in 75 districts where COVID cases had occurred as well as all major cities.Further, on 24 March, the prime minister ordered a nationwide lockdown for 21 days, affecting the entire 1.3 billion population of India.**

**The transmission escalated during March, after several cases were reported all over the country, most of which were linked to people with a travel history to affected countries. On 12 March, a 76-year-old man who had returned from Saudi Arabia became the first victim of the virus in the country.On 4 March, 22 new cases came to light, including those of an Italian tourist group with 14 infected members.But number of cases start increasing dramtically after 19th March, but in the month of April it has been its peak.**

**Experts suggest the number of infections could be much higher as India's testing rates are among the lowest in the world.The infection rate of COVID-19 in India is reported to be 1.7, significantly lower than in the worst affected countries.**

Source: Wikipedia

# Library

In [ ]:
import numpy as np 
import pandas as pd 
import os
from IPython.display import Markdown
from datetime import timedelta
import json, requests
from datetime import datetime
import glob
import requests 
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objs as go
import folium
%matplotlib inline
import seaborn as sns
sns.set()
import pycountry
from plotly.offline import init_notebook_mode, iplot 
import plotly.offline as py
import plotly.express as ex
from plotly.offline import download_plotlyjs,init_notebook_mode,plot,iplot
py.init_notebook_mode(connected=True)
import folium 
from folium import plugins
plt.style.use("seaborn-talk")
plt.rcParams['figure.figsize'] = 8, 5
plt.rcParams['image.cmap'] = 'viridis'
from fbprophet import Prophet
pd.set_option('display.max_rows', None)
from math import sin, cos, sqrt, atan2, radians
import warnings
warnings.filterwarnings("ignore")
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Uploading Data

In [ ]:
def get_distance_between_lats_lons(lat1,lon1,lat2,lon2):
# approximate radius of earth in km
        R = 6373
        lat1 = radians(lat1)
        lon1 = radians(lon1)
        lat2 = radians(lat2)
        lon2 = radians(lon2)
        dlon = lon2 - lon1
        dlat = lat2 - lat1
        a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
        c = 2 * atan2(sqrt(a), sqrt(1 - a))
        distance = R * c
        return(distance)
city_wise_coordinates= pd.read_csv('../input/indian-postal-codes/IndiaPostalCodes - IndiaPostalCodes.csv')
city_wise_coordinates['City'] = city_wise_coordinates['City'].str.upper()
city_wise_coordinates['District'] = city_wise_coordinates['District'].str.upper()
city_wise_coordinates['State'] = city_wise_coordinates['State'].str.upper()
district_wise_pin_states= city_wise_coordinates.groupby('District')['PIN','State'].agg(lambda x:x.value_counts().index[0])
district_wise_lat_lng= city_wise_coordinates.groupby('District')['Lat','Lng'].agg(pd.Series.mean)
district_wise_data_geonames= district_wise_pin_states.merge(district_wise_lat_lng,left_on='District', right_on='District', how= 'inner').reset_index()
dfp= pd.read_json("https://api.covid19india.org/raw_data.json")## data from covid19india.org
df3= []
for row in range(0,dfp.shape[0]):
    df1= dfp['raw_data'][row]
    df2=pd.DataFrame(df1.items()).set_index(0)
    df3.append(df2.T)           
appended_data = pd.concat(df3, sort=False)
appended_data.replace(r'^\s*$', np.nan, regex=True, inplace = True) 
appended_data.rename(columns={'detectedcity':'City'}, inplace=True)
appended_data.rename(columns={'detecteddistrict':'District'}, inplace=True)
appended_data.rename(columns={'detectedstate':'State'}, inplace=True)
appended_data['City'] = appended_data['City'].str.upper()
appended_data['District'] = appended_data['District'].str.upper()
appended_data['State'] = appended_data['State'].str.upper()
appended_data= appended_data.dropna(thresh=3)
district_wise_counts= appended_data.groupby('District').agg({'patientnumber': 'count'})
district_wise_counts.rename(columns={'patientnumber':'d_patient_counts'}, inplace=True)
district_wise_counts =district_wise_counts.reset_index()
district_wise_counts['District'] = district_wise_counts['District'].str.upper()
corona_db_with_latlng= district_wise_counts.merge(district_wise_data_geonames, left_on='District', right_on='District', how= 'inner')
corona_db_with_latlng.rename(columns={'d_patient_counts':'Num_Positive_cases'}, inplace=True)
def get_idx_distance_from_query_locations(q_lat, q_lng, corona_db_with_latlng):
    dist_array=[]
    for index, row in corona_db_with_latlng.iterrows():
        dist= int(get_distance_between_lats_lons(q_lat,q_lng,row['Lat'],row['Lng']))
        dist_array.append(dist)  
    minpos = dist_array.index(min(dist_array)) 
    mindist= dist_array[minpos]
    cases= corona_db_with_latlng.loc[minpos,'Num_Positive_cases']
    location= corona_db_with_latlng.loc[minpos,'District']
    state= corona_db_with_latlng.loc[minpos,'State']
    Lats= corona_db_with_latlng.loc[minpos,'Lat']
    Lngs= corona_db_with_latlng.loc[minpos,'Lng']
    return(mindist, cases, location, state)
def get_nearest_covid19_stats(query_info,corona_db_with_latlng):
    if query_info.PIN.iloc[1] in corona_db_with_latlng['PIN'].values:
        mindist= 2
        Lat= corona_db_with_latlng.loc[corona_db_with_latlng.PIN==query_info.PIN.iloc[1], 'Lat'];
        Lng= corona_db_with_latlng.loc[corona_db_with_latlng.PIN==query_info.PIN.iloc[1], 'Lng'];
        mindist= int(get_distance_between_lats_lons(query_info.Lat.iloc[1] ,query_info.Lng.iloc[1], Lat,Lng))
        cases= corona_db_with_latlng.loc[corona_db_with_latlng.PIN==query_info.PIN.iloc[1], 'Num_Positive_cases']
        district= corona_db_with_latlng.loc[corona_db_with_latlng.PIN==query_info.PIN.iloc[1], 'District']
        state= corona_db_with_latlng.loc[corona_db_with_latlng.PIN==query_info.PIN.iloc[1], 'State']
        print("The nearest location with COVID-19 from your PIN is in your own Postal Location with {} number of positive cases".format(cases.values[0]))
        print("Location: {} , {}".format(district.values[0].upper(), state.values[0].upper()))
    else:
        (mindist, cases, district, state) = get_idx_distance_from_query_locations(query_info.Lat.iloc[1] ,query_info.Lng.iloc[1], corona_db_with_latlng)  
        print("The nearest location with COVID-19 from your PIN is within {} km with {} number of positive cases".format(mindist, cases))
        print("Location: {} , {}".format(district.upper(), state.upper()))

In [ ]:
link = 'https://www.mohfw.gov.in/'
req = requests.get(link)
soup = BeautifulSoup(req.content, "html.parser")
thead = soup.find_all('thead')[-1]
head = thead.find_all('tr')
tbody = soup.find_all('tbody')[-1]
body = tbody.find_all('tr')
head_rows = []
body_rows = []
for tr in head:
    td = tr.find_all(['th', 'td'])
    row = [i.text for i in td]
    head_rows.append(row)   
for tr in body:
    td = tr.find_all(['th', 'td'])
    row = [i.text for i in td]
    body_rows.append(row)
df_bs = pd.DataFrame(body_rows[:len(body_rows)-1], columns=head_rows[0])   
df_bs.drop('S. No.', axis=1, inplace=True)
#To remove last raw
df_bs.drop(df_bs.tail(1).index,axis = 0,inplace=True)
now  = datetime.now()
df_bs['Date'] = now.strftime("%m/%d/%Y") 
df_bs['Date'] = pd.to_datetime(df_bs['Date'], format='%m/%d/%Y')

In [ ]:
locations = {
    "Kerala" : [10.8505,76.2711],
    "Maharashtra" : [19.7515,75.7139],
    "Karnataka": [15.3173,75.7139],
    "Telangana": [18.1124,79.0193],
    "Uttar Pradesh": [26.8467,80.9462],
    "Rajasthan": [27.0238,74.2179],
    "Gujarat":[22.2587,71.1924],
    "Delhi" : [28.7041,77.1025],
    "Punjab":[31.1471,75.3412],
    "Tamil Nadu": [11.1271,78.6569],
    "Haryana": [29.0588,76.0856],
    "Madhya Pradesh":[22.9734,78.6569],
    "Jammu and Kashmir":[33.7782,76.5762],
    "Ladakh": [34.1526,77.5770],
    "Andhra Pradesh":[15.9129,79.7400],
    "West Bengal": [22.9868,87.8550],
    "Bihar": [25.0961,85.3131],
    "Chhattisgarh":[21.2787,81.8661],
    "Chandigarh":[30.7333,76.7794],
    "Uttarakhand":[30.0668,79.0193],
    "Himachal Pradesh":[31.1048,77.1734],
    "Goa": [15.2993,74.1240],
    "Odisha":[20.9517,85.0985],
    "Andaman and Nicobar Islands": [11.7401,92.6586],
    "Puducherry":[11.9416,79.8083],
    "Manipur":[24.6637,93.9063],
    "Mizoram":[23.1645,92.9376],
    "Assam":[26.2006,92.9376],
    "Meghalaya":[25.4670,91.3662],
    "Tripura":[23.9408,91.9882],
    "Arunachal Pradesh":[28.2180,94.7278],
    "Jharkhand" : [23.6102,85.2799],
    "Nagaland": [26.1584,94.5624],
    "Sikkim": [27.5330,88.5122],
    "Dadra and Nagar Haveli":[20.1809,73.0169],
    "Lakshadweep":[10.5667,72.6417],
    "Daman and Diu":[20.4283,72.8397]   
}

In [ ]:
lat = {'Delhi':28.7041,
       'Haryana':29.0588,
       'Kerala':10.8505,
       'Rajasthan':27.0238,
       'Telengana':18.1124,
       'Uttar Pradesh':26.8467,
       'Ladakh':34.2996,
       'Tamil Nadu':11.1271,
       'Jammu and Kashmir':33.7782,
       'Punjab':31.1471,
       'Karnataka':15.3173,
       'Maharashtra':19.7515,
       'Andhra Pradesh':15.9129, 
       'Odisha':20.9517, 
       'Uttarakhand':30.0668, 
       'West Bengal':22.9868, 
       'Puducherry': 11.9416, 
       'Chandigarh': 30.7333, 
       'Chhattisgarh':21.2787, 
       'Gujarat': 22.2587, 
       'Himachal Pradesh': 31.1048, 
       'Madhya Pradesh': 22.9734, 
       'Bihar': 25.0961, 
       'Manipur':24.6637,
       'Mizoram':23.1645,
        'Goa':15.2993,
     'Andaman and Nicobar Islands':11.7401,
      "Jharkhand" : 23.6102,
      'Arunachal Pradesh': 28.2180,
      'Assam' : 26.2006,
      'Tripura':23.9408}

long = {'Delhi':77.1025,
        'Haryana':76.0856,
        'Kerala':76.2711,
        'Rajasthan':74.2179,
        'Telengana':79.0193,
        'Uttar Pradesh':80.9462,
        'Ladakh':78.2932,
        'Tamil Nadu':78.6569,
        'Jammu and Kashmir':76.5762,
        'Punjab':75.3412,
        'Karnataka':75.7139,
        'Maharashtra':75.7139,
        'Andhra Pradesh':79.7400, 
        'Odisha':85.0985, 
        'Uttarakhand':79.0193, 
        'West Bengal':87.8550, 
        'Puducherry': 79.8083, 
        'Chandigarh': 76.7794, 
        'Chhattisgarh':81.8661, 
        'Gujarat': 71.1924, 
        'Himachal Pradesh': 77.1734, 
        'Madhya Pradesh': 78.6569, 
        'Bihar': 85.3131, 
        'Manipur':93.9063,
        'Mizoram':92.9376,
         'Goa':74.1240,
        "Jharkhand" : 85.2799,
       'Andaman and Nicobar Islands':92.6586,
       'Arunachal Pradesh' :94.7278,
        'Assam' : 92.9376,
        'Tripura':91.9882
       }
df_bs['Latitude'] = df_bs['Name of State / UT'].map(lat)
df_bs['Longitude'] = df_bs['Name of State / UT'].map(long)
df_bs['Total cases'] = df_bs.iloc[:,1]

In [ ]:
# complete data

file_name = now.strftime("%Y_%m_%d")+'.csv'
file_loc = ''
df_bs.to_csv(file_loc + file_name, index=False)
loc = ""
files = glob.glob(loc+'2020*.csv')
dfs = []
for i in files:
    df_temp = pd.read_csv(i)
    df_temp = df_temp.rename(columns={'Cured':'Cured/Discharged'})
    df_temp = df_temp.rename(columns={'Cured/Discharged':'Cured/Discharged/Migrated'})
    dfs.append(df_temp)
    
# print(dfs)

complete_data = pd.concat(dfs, ignore_index=True).sort_values(['Date'], ascending=True).reset_index(drop=True)
complete_data['Date'] = pd.to_datetime(complete_data['Date'])
complete_data = complete_data.sort_values(['Date', 'Name of State / UT']).reset_index(drop=True)
cols = ['Total cases','Cured/Discharged/Migrated', 'Death']
tot = complete_data.iloc[:,1]
complete_data[cols] = complete_data[cols].fillna(0).astype('int')

In [ ]:
data = pd.DataFrame(complete_data)
covid_19_India = pd.read_csv('../input/covid19-in-india/covid_19_india.csv')
df = pd.read_csv('../input/covid19-corona-virus-india-dataset/complete.csv', parse_dates=['Date'])
p_df = pd.read_csv('../input/covid19-corona-virus-india-dataset/patients_data.csv')
p_df['date_announced'] = pd.to_datetime(p_df['date_announced'], errors = 'coerce')
p_df['date_announced'] = pd.to_datetime(p_df['date_announced'], format='%d/%m/%Y')
p_df['status_change_date'] = pd.to_datetime(p_df['status_change_date'], format='%d/%m/%Y')
p_df['nationality'] = p_df['nationality'].replace('Indian', 'India')
india_data_json = requests.get('https://api.rootnet.in/covid19-in/unofficial/covid19india.org/statewise').json()
df_india = pd.io.json.json_normalize(india_data_json['data']['statewise'])
df_india = df_india.set_index("state")
test1 = pd.read_csv('../input/icmr-testing-data/testing data.csv')

# Data Cleaning

In [ ]:
covid_19_India.drop(['ConfirmedIndianNational','ConfirmedForeignNational'],axis=1, inplace = True)
covid_19_India['Active']= covid_19_India['Confirmed']- (covid_19_India['Cured'] + covid_19_India['Deaths'])

In [ ]:
data['Active cases'] = data['Total cases'] - (data['Cured/Discharged/Migrated'] + data['Death'])
data.to_csv('state_wise_data.csv',index=False)

In [ ]:
df['Name of State / UT'] = df['Name of State / UT'].str.replace('Union Territory of ', '')
df = df[['Date', 'Name of State / UT', 'Latitude', 'Longitude', 'Total Confirmed cases', 'Death', 'Cured/Discharged/Migrated']]
df.columns = ['Date', 'State/UT', 'Latitude', 'Longitude', 'Confirmed', 'Deaths', 'Cured']

for i in ['Confirmed', 'Deaths', 'Cured']:
    df[i] = df[i].astype('int')
    
df['Active'] = df['Confirmed'] - df['Deaths'] - df['Cured']
df['Mortality rate'] = df['Deaths']/df['Confirmed']
df['Recovery rate'] = df['Cured']/df['Confirmed']

df = df[['Date', 'State/UT', 'Latitude', 'Longitude', 'Confirmed', 'Active', 'Deaths', 'Mortality rate', 'Cured', 'Recovery rate']]

In [ ]:
latest = df[df['Date']==max(df['Date'])]

# days
latest_day = max(df['Date'])
day_before = latest_day - timedelta(days = 1)

# state and total cases 
latest_day_df = df[df['Date']==latest_day].set_index('State/UT')
day_before_df = df[df['Date']==day_before].set_index('State/UT')
temp = pd.merge(left = latest_day_df, right = day_before_df, on='State/UT', suffixes=('_lat', '_bfr'), how='outer')
latest_day_df['New cases'] = temp['Confirmed_lat'] - temp['Confirmed_bfr']
latest = latest_day_df.reset_index()
latest.fillna(1, inplace=True)
latest.to_csv('statewise_data_with_new_cases.csv')

In [ ]:
date_wise_data = covid_19_India[['State/UnionTerritory',"Date","Confirmed","Deaths","Cured",'Active']]
date_wise_data['Date'] = date_wise_data['Date'].apply(pd.to_datetime, dayfirst=True)
date_wise_data = date_wise_data.groupby(["Date"]).sum().reset_index()
def formatted_text(string):
    display(Markdown(string))
date_wise_data.to_csv('date_wise_data.csv')
#date_wise_data.head()

In [ ]:
df_india["Lat"] = ""
df_india["Long"] = ""
for index in df_india.index :
    df_india.loc[df_india.index == index,"Lat"] = locations[index][0]
    df_india.loc[df_india.index == index,"Long"] = locations[index][1]

In [ ]:
test1["positive_ratio"]= np.round(100*test1["totalPositiveCases"]/test1["totalSamplesTested"],2)
test1["perday_positive"] = test1["totalPositiveCases"].diff()
test1["perday_tests"] = test1["totalSamplesTested"].diff()
test1["positive_ratio"]= np.round(100*test1["perday_positive"]/test1["perday_tests"],2)
test1.fillna(0)
test1.to_csv('ICMR_Testing_Data.csv')
#test1.head()

# Enter your location as an Indian PINCODE to Find the nearest corona positive location and the number of cases.

In [ ]:
print('Enter your Indian Pincode number to Find the nearest corona positive location and the number of cases with risk factor (low, moderate, high)')
query_pincode = input("Enter you PINCODE to know status  : ") 
g = int(query_pincode)
if g in city_wise_coordinates.PIN.values:
    query_info= city_wise_coordinates[city_wise_coordinates.PIN == int(g)]
    get_nearest_covid19_stats(query_info,corona_db_with_latlng)
else:
    print('You entered an Invalid PIN')


# **Overall view of the situation**

In [ ]:
total_tested = test1["totalSamplesTested"][-1:].values[0]
total_positive = test1["totalPositiveCases"][-1:].values[0]
positivecase_ratio = total_positive * 100 / total_tested
pcr = float("{:.2f}".format(positivecase_ratio))
print('Total Number of people tested :', total_tested)
print('Total Number of positive cases :',total_positive)
print('Positive case ratio cases:',pcr)

In [ ]:
#Overall 
ac= data['Active cases'].sum()
rvd = data['Cured/Discharged/Migrated'].sum()
dth = data['Death'].sum()
fig = go.Figure(data=[go.Pie(labels=['Active Cases','Cured','Death'],
                             values= [ac,rvd,dth],hole =.3)])
fig.update_traces(hoverinfo='label+percent', textinfo='value', textfont_size=20,
                  marker=dict(colors=['#263fa3', '#2fcc41','#cc3c2f'], line=dict(color='#FFFFFF', width=2)))
fig.update_layout(title_text='Current Situation in India according www.mohfw.gov.in',plot_bgcolor='rgb(275, 270, 273)')
fig.show()

In [ ]:
#Overall 
ac= df_india['active'].sum()
rvd = df_india['recovered'].sum()
dth = df_india['deaths'].sum()
fig = go.Figure(data=[go.Pie(labels=['Active Cases','Cured','Death'],
                             values= [ac,rvd,dth],hole =.3)])
fig.update_traces(hoverinfo='label+percent', textinfo='value', textfont_size=20,
                  marker=dict(colors=['#263fa3', '#2fcc41','#cc3c2f'], line=dict(color='#FFFFFF', width=2)))
fig.update_layout(title_text='Current Situation in India according www.covid19india.org',plot_bgcolor='rgb(275, 270, 273)')
fig.show()

# Gender wise insights of COVID-19 cases in india

In [ ]:
tempa = p_df[['current_status','gender']].dropna()
actv = tempa[tempa['current_status'] == 'Hospitalized']
a_f = len(actv[actv['gender'] == 'F'])
a_m = len(actv[actv['gender'] == 'M'])
fig = go.Figure(data=[go.Pie(labels=['Male','Female'],
                             values= [a_m,a_f],hole =.3)])
fig.update_traces(hoverinfo='label+percent', textinfo='value', textfont_size=20,
                  marker=dict(colors=['#263fa3', '#d461bf'], line=dict(color='#FFFFFF', width=2)))
fig.update_layout(title_text='Gender wise Active Cases',plot_bgcolor='rgb(275, 270, 273)')
fig.show()


In [ ]:
tempr = p_df[['current_status','gender']].dropna()
rcvd = tempr[tempr['current_status'] == 'Recovered']
r_f = len(rcvd[rcvd['gender'] == 'F'])
r_m = len(rcvd[rcvd['gender'] == 'M'])
fig = go.Figure(data=[go.Pie(labels=['Male','Female'],
                             values= [r_m,r_f],hole =.3)])
fig.update_traces(hoverinfo='label+percent', textinfo='value', textfont_size=20,
                  marker=dict(colors=['#263fa3', '#d461bf'], line=dict(color='#FFFFFF', width=2)))
fig.update_layout(title_text='Gender wise Recovered Cases(Numbers are inaccurate due to missing values)',
plot_bgcolor='rgb(275, 270, 273)')
fig.show()

In [ ]:
#option
temp = p_df[['current_status','gender']].dropna()
dd = temp[temp['current_status'] == 'Deceased']
d_f = len(dd[dd['gender'] == 'F'])
d_m = len(dd[dd['gender'] == 'M'])
fig = go.Figure(data=[go.Pie(labels=['Male','Female'],
                             values= [d_m,d_f],hole =.3)])
fig.update_traces(hoverinfo='label+percent', textinfo='value', textfont_size=20,
                  marker=dict(colors=['#263fa3','#d461bf'], line=dict(color='#FFFFFF', width=2)))
fig.update_layout(title_text='Gender wise Deaths (Numbers are inaccurate due to missing values)',plot_bgcolor='rgb(275, 270, 273)')
fig.show()

In [ ]:
temp = p_df[['status_change_date','age_bracket','current_status','gender','detected_state']].dropna()
rec = temp[temp['current_status'] == 'Recovered'].drop('current_status',axis =1).sort_values('status_change_date',ascending=True)
rec_x = rec['age_bracket'].astype(int)
rec_y = rec['gender']
fig = px.histogram(x=rec_x,color =rec_y,orientation = 'v',
                   title='Age wise Recovered cases in Male and Female (Numbers are inaccurate due to missing values)')
fig.update_layout(barmode='stack')
fig.update_layout(plot_bgcolor='rgb(275, 270, 273)',yaxis_title='Deaths',xaxis_title='Age Group')
fig.show()

In [ ]:
temp = p_df.copy()
temp = p_df[['status_change_date','age_bracket','current_status','gender','detected_state']].dropna()
dea = temp[temp['current_status'] == 'Deceased'].drop('current_status',axis =1).sort_values('status_change_date',ascending=True)
xaxis = dea['age_bracket'].astype(int)
yaxis = dea['gender']
fig = px.histogram(x=xaxis,color =yaxis,orientation = 'v',
                   title='Age wise Deaths in Male and Female (Numbers are inaccurate due to missing values)')
fig.update_layout(barmode='stack')
fig.update_layout(plot_bgcolor='rgb(275, 270, 273)',yaxis_title='Deaths',xaxis_title='Age Group')
fig.show()

# **Trend of COVID-19 cases in India**

In [ ]:
import plotly.express as px
fig = px.bar(test1, x="day", y="perday_tests", barmode='group',height=500,color = "perday_tests",
             orientation = 'v',
            color_discrete_sequence = px.colors.sequential.Plasma_r)
fig.update_layout(title_text='Number of COVID-19 test conducted everyday',plot_bgcolor='rgb(275, 270, 273)')
fig.update_layout(barmode='stack')
fig.update_layout(plot_bgcolor='rgb(275, 270, 273)',yaxis_title='Tests',xaxis_title='Date')
fig.show()

In [ ]:
fig = go.Figure(data=[
go.Bar(name='Tested', y=test1['day'], x=test1['perday_tests'], orientation='h'),
go.Bar(name='Positive', y=test1['day'], x=test1['perday_positive'], orientation='h')])
fig.update_layout(barmode='stack',width=500, height=600)
fig.update_traces(textposition='inside')
fig.update_layout(uniformtext_minsize=8, uniformtext_mode='hide')
fig.update_layout(title_text='Number of people tested and positive among on different date',
                  plot_bgcolor='rgb(275, 270, 273)')
fig.show()

In [ ]:
perday = covid_19_India.groupby(['Date'])['Confirmed'].sum().reset_index().sort_values('Confirmed',ascending = True)
perday['New Daily Cases'] = perday['Confirmed'].sub(perday['Confirmed'].shift())
perday['New Daily Cases'].iloc[0] = perday['Confirmed'].iloc[0]
perday['New Daily Cases'] = perday['New Daily Cases'].astype(int)
# New COVID-19 cases reported daily in India
import plotly.express as px
fig = px.bar(perday, x="Date", y="New Daily Cases", barmode='group',height=500,color = "New Daily Cases",
            color_discrete_sequence = px.colors.sequential.Plasma_r)
fig.update_layout(title_text='New COVID-19 cases reported daily in India',plot_bgcolor='rgb(275, 270, 273)')
fig.show()
perday.to_csv('perday_new_cases.csv')

In [ ]:
temp = df.copy()
temp = df.groupby('Date')['Confirmed', 'Deaths', 'Cured'].sum().reset_index()
fig = px.scatter(temp, x="Date", y="Confirmed", color="Confirmed",
                 size='Confirmed', hover_data=['Confirmed'],
                 color_discrete_sequence = ex.colors.cyclical.IceFire)
fig.update_layout(title_text='Trend of Daily Coronavirus Cases in India',
                  plot_bgcolor='rgb(275, 270, 273)',width=600, height=600)
fig.show()

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=date_wise_data['Date'], y=date_wise_data['Confirmed'],
                    mode='lines+markers',marker_color='blue',name='Total Cases'))
fig.add_trace(go.Scatter(x=date_wise_data['Date'],y=date_wise_data['Active'], 
                mode='lines+markers',marker_color='purple',name='Active'))
fig.add_trace(go.Scatter(x=date_wise_data['Date'], y=date_wise_data['Cured'],
                mode='lines+markers',marker_color='green',name='Recovered'))
fig.add_trace(go.Scatter(x=date_wise_data['Date'], y=date_wise_data['Deaths'], 
                mode='lines+markers',marker_color='red',name='Deaths'))
fig.update_layout(title_text='Trend of Weekly Coronavirus Cases in India',plot_bgcolor='rgb(275, 270, 273)',width=600, height=600)
fig.show()

In [ ]:
cnf = '#263fa3' # confirmed - blue
act = '#fe9801' # active case - yellow
rec = '#21bf73' # recovered - green
dth = '#de260d' # death - red
tmp = date_wise_data.melt(id_vars="Date",value_vars=['Deaths','Cured' ,'Active','Confirmed'],
                 var_name='Case',value_name='Count')
fig = px.area(tmp, x="Date", y="Count",color='Case',
              title='Trend Over Weeks',color_discrete_sequence = [dth,rec,act,cnf])
fig.update_layout(plot_bgcolor='rgb(275, 270, 273)',width=550, height=600)
fig.show()

In [ ]:
df_india_data = df[['Date', 'State/UT','Confirmed','Cured','Deaths','Active']]

In [ ]:
spread = df_india_data.groupby(['Date', 'State/UT'])['Confirmed'].sum().reset_index().sort_values('Confirmed', ascending=True)
fig = px.area(spread, x="Date", y="Confirmed",color='State/UT',title='State Wise Spread over time',height=500,
              color_discrete_sequence = ex.colors.cyclical.Edge)
fig.update_layout(plot_bgcolor='rgb(275, 270, 273)',width=700, height=600)

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=test1['day'], y=test1['positive_ratio'],
                    mode='lines+markers',marker_color='blue'))
fig.update_layout(title_text = 'Trend of Positive case ratio from tested people of India')
fig.update_layout(plot_bgcolor='rgb(275, 270, 273)',width=600, height=600)
fig.show()

In [ ]:
temp = date_wise_data.copy()
temp['Recovery Rate'] = temp['Cured']/temp['Confirmed']*100
fig = go.Figure()
fig.add_trace(go.Scatter(x=temp['Date'], y=temp['Recovery Rate'],
                    mode='lines+markers',marker_color='green'))
fig.update_layout(title_text = 'Trend of Recovery Rate of India')
fig.update_layout(plot_bgcolor='rgb(275, 270, 273)',width=600, height=600)
fig.show()

In [ ]:
temp = date_wise_data.copy()
temp['Mortality Rate'] = temp['Deaths']/temp['Confirmed']*100
fig = go.Figure()
fig.add_trace(go.Scatter(x=temp['Date'], y=temp['Mortality Rate'],mode='lines+markers',marker_color='red'))
fig.update_layout(title_text = 'Trend of Mortality Rate of India')
fig.update_layout(plot_bgcolor='rgb(275, 270, 273)',width=600, height=600)
fig.show()

In [ ]:
agegroup = pd.read_csv('../input/covid19-in-india/AgeGroupDetails.csv')
fig = go.Figure()
fig.add_trace(go.Scatter(x=agegroup['AgeGroup'],y=agegroup['TotalCases'],line_shape='spline',fill='tonexty',fillcolor = 'orange')) 
fig.update_layout(title="Age wise Confirmed Case Trend in India",yaxis_title="Total Number of cases",xaxis_title="Age Group")
fig.update_layout(plot_bgcolor='rgb(275, 270, 273)',width=600,height=600)
fig.show()

# **Covid-19 State Wise insight**

In [ ]:
temp = data.sort_values('Total cases', ascending=True)
fig = go.Figure(data=[
go.Bar(name='Active', y=temp['Name of State / UT'], x=temp['Active cases'], orientation='h'),
go.Bar(name='Death', y=temp['Name of State / UT'], x=temp['Death'], orientation='h'),
go.Bar(name='Cured', y=temp['Name of State / UT'], x=temp['Cured/Discharged/Migrated'], orientation='h')])
fig.update_layout(barmode='stack',width=600, height=800)
#fig.update_traces(textposition='inside')
fig.update_layout(uniformtext_minsize=8, uniformtext_mode='hide')
fig.update_layout(title_text='Active Cases,Cured,Deaths in Different States of India',plot_bgcolor='rgb(275, 270, 273)')
fig.show()

In [ ]:
#colors 
'''
color_discrete_sequence = px.colors.sequential.Plasma_r, ex.colors.cyclical.IceFire, ex.colors.cyclical.Edge'
'''

In [ ]:
temp = data.copy()
temp = data.sort_values('Total cases', ascending=False)
state_order = temp['Name of State / UT']
fig = px.bar(temp,x="Total cases", y="Name of State / UT", color='Name of State / UT', 
             title='State Wise Confirmed Cases', orientation='h', text='Total cases', 
             height=900,color_discrete_sequence = ex.colors.cyclical.IceFire)
fig.update_layout(plot_bgcolor='rgb(275, 270, 273)')
fig.show()

In [ ]:
temp = data.copy()
temp = data[data['Cured/Discharged/Migrated']>0].sort_values('Cured/Discharged/Migrated',ascending=False)
state_order = temp['Name of State / UT']
fig = px.bar(temp,x="Cured/Discharged/Migrated", y="Name of State / UT", color='Name of State / UT',
             title='State wise Cured/Discharged/Migrated cases', orientation='h', 
             text='Cured/Discharged/Migrated', 
             height=600,color_discrete_sequence = ex.colors.cyclical.Phase)
fig.update_layout(plot_bgcolor='rgb(275, 270, 273)')
fig.show()

In [ ]:
temp = data.copy()
temp = data[data['Death']>0].sort_values('Death',ascending=False)
state_order = temp['Name of State / UT']
fig = px.bar(temp,x="Death", y="Name of State / UT", color='Name of State / UT',
             title='State wise Deaths', orientation='h', 
             text='Death', 
             height=600,color_discrete_sequence = px.colors.sequential.Plasma_r)
fig.update_layout(plot_bgcolor='rgb(275, 270, 273)')
fig.show()

# Demographics Visualisation

In [ ]:
india = folium.Map(location=[20.5937, 78.9629], zoom_start=4,max_zoom=6,min_zoom=4,tiles='cartodbpositron',height = 700)
for i in range(0,len(df_india[df_india['confirmed']>0].index)):
    folium.Circle(
        location=[df_india.iloc[i]['Lat'], df_india.iloc[i]['Long']],
        tooltip = "<h5 style='text-align:center;font-weight: bold'>"+df_india.iloc[i].name+"</h5>"+
                    "<hr style='margin:10px;'>"+
                    "<ul style='color: #444;list-style-type:circle;align-item:left;padding-left:20px;padding-right:20px'>"+
        "<li>Confirmed: "+str(df_india.iloc[i]['confirmed'])+"</li>"+
        "<li>Active:   "+str(df_india.iloc[i]['active'])+"</li>"+
        "<li>Recovered:   "+str(df_india.iloc[i]['recovered'])+"</li>"+
        "<li>Deaths:   "+str(df_india.iloc[i]['deaths'])+"</li>"+
        "<li>Mortality Rate:   "+str(np.round(df_india.iloc[i]['deaths']/(df_india.iloc[i]['confirmed']+1)*100,2))+"</li>"+
        "</ul>",
        radius=(int(np.log2(df_india.iloc[i]['confirmed']+1)))*9000,
        color='red',
        fill_color='green',
        fill=True).add_to(india)
india


**COVID-19 Cases thtough time in different parts of india**

In [ ]:
from plotly.offline import init_notebook_mode
import plotly.graph_objs as go
init_notebook_mode(connected=True)
tmp = df.copy()
tmp['Date'] = tmp['Date'].dt.strftime('%Y/%m/%d')
fig = px.scatter_geo(tmp,lat="Latitude", lon="Longitude", color='Confirmed', size='Confirmed', projection="natural earth",
                     hover_name="State/UT", scope='asia', animation_frame="Date",
                     color_continuous_scale=px.colors.diverging.curl,center={'lat':20, 'lon':78}, 
                     range_color=[0, max(tmp['Confirmed'])])
fig.update_layout(plot_bgcolor='rgb(275, 270, 273)')
fig.show()
tmp.to_csv('covid_19_india.csv')

# Enter you PINCODE and Find the nearest corona positive location and the number of cases with risk factor (low, moderate, high)

In [ ]:
print('Enter your Indian Pincode number to Find the nearest corona positive location and the number of cases with risk factor (low, moderate, high)')
query_pincode = input("Enter you PINCODE to know status  : ") 
g = int(query_pincode)

if g in city_wise_coordinates.PIN.values:
    query_info= city_wise_coordinates[city_wise_coordinates.PIN == int(g)]
    get_nearest_covid19_stats(query_info,corona_db_with_latlng)
else:
    print('You entered an Invalid PIN')


# Together We can do this.

![](https://thespinoff.co.nz/wp-content/uploads/2020/03/Covid-19-curves-graphic-social-v3.gif)

Image Source: https://thespinoff.co.nz/wp-content/uploads/2020/03/Covid-19-curves-graphic-social-v3.gif

**Note: This work is highly inspired from few other kaggle kernels , github sources and other data science resources. Any traces of replications, which may appear , is purely co-incidental. Due respect & credit to all my fellow kagglers.**